<a href="https://colab.research.google.com/github/AmbiguousError/orca_watch/blob/main/devonport_orca_detect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install libraries required

In [2]:
!pip install yt-dlp ultralytics opencv-python matplotlib



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.0/800.0 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 7.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
 

YOLOv8 Model: Download a pre-trained YOLOv8 object detection model.
Deets here: https://github.com/ultralytics/ultralytics



In [3]:
! yolo task=detect mode=predict model=yolov8n.pt


100% 6.25M/6.25M [00:00<00:00, 180MB/s]
WARNING ⚠️ 'source' argument is missing. Using default 'source=/usr/local/lib/python3.10/dist-packages/ultralytics/assets'.
Ultralytics YOLOv8.2.52 🚀 Python-3.10.12 torch-2.3.0+cu121 CPU (Intel Xeon 2.20GHz)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs

image 1/2 /usr/local/lib/python3.10/dist-packages/ultralytics/assets/bus.jpg: 640x480 4 persons, 1 bus, 1 stop sign, 292.7ms
image 2/2 /usr/local/lib/python3.10/dist-packages/ultralytics/assets/zidane.jpg: 384x640 2 persons, 1 tie, 169.6ms
Speed: 9.8ms preprocess, 231.2ms inference, 16.6ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict
💡 Learn more at https://docs.ultralytics.com/modes/predict


check feed formats available

In [4]:
!yt-dlp -F https://www.youtube.com/watch?v=6VHnY6ixjLc


[youtube] Extracting URL: https://www.youtube.com/watch?v=6VHnY6ixjLc
[youtube] 6VHnY6ixjLc: Downloading webpage
[youtube] 6VHnY6ixjLc: Downloading ios player API JSON
[youtube] 6VHnY6ixjLc: Downloading m3u8 information
[youtube] 6VHnY6ixjLc: Downloading m3u8 information
[info] Available formats for 6VHnY6ixjLc:
ID  EXT RESOLUTION FPS │   TBR PROTO │ VCODEC        VBR ACODEC     MORE INFO
─────────────────────────────────────────────────────────────────────────────
233 mp4 audio only     │       m3u8  │ audio only        unknown    Default
234 mp4 audio only     │       m3u8  │ audio only        unknown    Default
269 mp4 256x144     15 │  290k m3u8  │ avc1.42C00B  290k video only
91  mp4 256x144     15 │  290k m3u8  │ avc1.42c00b       mp4a.40.5
229 mp4 426x240     30 │  546k m3u8  │ avc1.4D4015  546k video only
92  mp4 426x240     30 │  546k m3u8  │ avc1.4d4015       mp4a.40.5
230 mp4 640x360     30 │ 1210k m3u8  │ avc1.4D401E 1210k video only
93  mp4 640x360     30 │ 1210k m3u8  │ a

run script, will apparently show pics of orca in cell when viewed ... should setup test

In [ ]:
import cv2
import yt_dlp
import subprocess
import os
from ultralytics import YOLO
import matplotlib.pyplot as plt

def detect_orcas(video_filename, model_path="yolov8n.pt"):
    """Detects orcas in a video file."""
    model = YOLO(model_path)
    cap = cv2.VideoCapture(video_filename)
    while cap.isOpened():
        success, frame = cap.read()
        if success:
            results = model(frame)
            for result in results:
                for box in result.boxes:
                    if result.names[box.cls[0]] == "orca":
                        x1, y1, x2, y2 = map(int, box.xyxy[0])
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                        label = f"{result.names[box.cls[0]]} {box.conf[0]:.2f}"
                        cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

            # Display the frame using Matplotlib
            plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))  # Convert to RGB for Matplotlib
            plt.title("Orca Detection")
            plt.show()

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break
    cap.release()
    cv2.destroyAllWindows()

def process_live_stream(live_stream_url, model_path="yolov8n.pt"):
    """Processes a live stream in chunks."""
    temp_filename = "temp.ts"  # Change to .ts for MPEG-TS
    ydl_opts = {
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best',
        'verbose': True,
        'noplaylist': True,
        'merge_output_format': 'mp4'  # Merge audio and video into mp4 after download
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(live_stream_url)
        video_filename = ydl.prepare_filename(info)

        while True:
            # Record a segment of the stream
            ffmpeg_cmd = f"ffmpeg -protocol_whitelist file,http,https,tcp,tls,crypto -i {video_filename} -c copy -bsf:a aac_adtstoasc -t 30 {temp_filename}"
            print(f"Recording segment: {ffmpeg_cmd}")

            try:
                subprocess.run(ffmpeg_cmd, shell=True, check=True)
            except subprocess.CalledProcessError as e:
                print(f"FFmpeg error: {e.stderr}")  # Print FFmpeg error
                break

            if os.path.exists(temp_filename):
                print(f"Processing segment: {temp_filename}")
                detect_orcas(temp_filename, model_path)
                os.remove(temp_filename)  # Remove the temporary file
            else:
                print("Error: Temporary file not created.")


if __name__ == "__main__":
    youtube_url = "https://www.youtube.com/watch?v=6VHnY6ixjLc"  # Replace with the actual live stream URL
    process_live_stream(youtube_url)


[debug] Encodings: locale UTF-8, fs utf-8, pref UTF-8, out UTF-8 (No ANSI), error UTF-8 (No ANSI), screen UTF-8 (No ANSI)
[debug] yt-dlp version stable@2024.07.09 from yt-dlp/yt-dlp [7ead7332a] (pip) API
[debug] params: {'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best', 'verbose': True, 'noplaylist': True, 'merge_output_format': 'mp4', 'compat_opts': set(), 'http_headers': {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.24 Safari/537.36', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', 'Accept-Language': 'en-us,en;q=0.5', 'Sec-Fetch-Mode': 'navigate'}}
[debug] Python 3.10.12 (CPython x86_64 64bit) - Linux-6.1.85+-x86_64-with-glibc2.35 (OpenSSL 3.0.2 15 Mar 2022, glibc 2.35)
[debug] exe versions: ffmpeg 4.4.2 (setts), ffprobe 4.4.2
[debug] Optional libraries: Cryptodome-3.20.0, brotli-1.1.0, certifi-2024.06.02, mutagen-1.47.0, requests-2.32.3, secretstorage-3.3.1, sqlite3-3

[youtube] Extracting URL: https://www.youtube.com/watch?v=6VHnY6ixjLc
[youtube] 6VHnY6ixjLc: Downloading webpage
[youtube] 6VHnY6ixjLc: Downloading ios player API JSON
[youtube] 6VHnY6ixjLc: Downloading m3u8 information
[youtube] 6VHnY6ixjLc: Downloading m3u8 information


[debug] Sort order given by extractor: quality, res, fps, hdr:12, source, vcodec:vp9.2, channels, acodec, lang, proto
[debug] Formats sorted by: hasvid, ie_pref, quality, res, fps, hdr:12(7), source, vcodec:vp9.2(10), channels, acodec, lang, proto, size, br, asr, vext, aext, hasaud, id


[info] 6VHnY6ixjLc: Downloading 1 format(s): 96


[debug] Invoking ffmpeg downloader on "https://manifest.googlevideo.com/api/manifest/hls_playlist/expire/1720601232/ei/MPaNZvbcHIuDi9oP9pKk0AI/ip/34.34.25.49/id/6VHnY6ixjLc.1/itag/96/source/yt_live_broadcast/requiressl/yes/ratebypass/yes/live/1/sgoap/gir%3Dyes%3Bitag%3D140/sgovp/gir%3Dyes%3Bitag%3D137/rqh/1/hdlc/1/hls_chunk_host/rr2---sn-5hne6nsk.googlevideo.com/xpc/EgVo2aDSNQ%3D%3D/playlist_duration/30/manifest_duration/30/spc/NO7bAWTAnkB0KolyDBKG05jHWZxIDHApAdQ0Lt6UlnlR0woYTRaZ_sLAu3Qpf10/vprv/1/playlist_type/DVR/mh/uo/mm/44/mn/sn-5hne6nsk/ms/lva/mv/m/mvi/2/pl/17/dover/11/pacing/0/keepalive/yes/mt/1720579474/sparams/expire,ei,ip,id,itag,source,requiressl,ratebypass,live,sgoap,sgovp,rqh,hdlc,xpc,playlist_duration,manifest_duration,spc,vprv,playlist_type/sig/AJfQdSswRgIhAIvMhXhjW5xFvrjp6a7-mKmxMGw5T7e_aIg1aHILdpBFAiEAiTo2iruzUMp-jNr2U-B0emz77KA7L13SI9akSWkV5vw%3D/lsparams/hls_chunk_host,mh,mm,mn,ms,mv,mvi,pl/lsig/AHlkHjAwRQIhAPFThyfwo4fM1yi_9thjgz7aG7sW3oPCmcKN45X3g7m2AiA_mGUPpWAk__L5D

[download] Destination: Hauraki Gulf Weather live stream (HaurakiGulfWeather.com) 2024-07-10 02_47 [6VHnY6ixjLc].mp4


[debug] ffmpeg command line: ffmpeg -y -loglevel verbose -headers 'User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.24 Safari/537.36
Accept: text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8
Accept-Language: en-us,en;q=0.5
Sec-Fetch-Mode: navigate
' -i https://manifest.googlevideo.com/api/manifest/hls_playlist/expire/1720601232/ei/MPaNZvbcHIuDi9oP9pKk0AI/ip/34.34.25.49/id/6VHnY6ixjLc.1/itag/96/source/yt_live_broadcast/requiressl/yes/ratebypass/yes/live/1/sgoap/gir%3Dyes%3Bitag%3D140/sgovp/gir%3Dyes%3Bitag%3D137/rqh/1/hdlc/1/hls_chunk_host/rr2---sn-5hne6nsk.googlevideo.com/xpc/EgVo2aDSNQ%3D%3D/playlist_duration/30/manifest_duration/30/spc/NO7bAWTAnkB0KolyDBKG05jHWZxIDHApAdQ0Lt6UlnlR0woYTRaZ_sLAu3Qpf10/vprv/1/playlist_type/DVR/mh/uo/mm/44/mn/sn-5hne6nsk/ms/lva/mv/m/mvi/2/pl/17/dover/11/pacing/0/keepalive/yes/mt/1720579474/sparams/expire,ei,ip,id,itag,source,requiressl,ratebypass,live,sgoap,sgovp,rqh,hdlc,xpc,playl